# Análisis de los diferentes modelos

## 1. Importación de librerías y modulos

In [1]:
#%pip install -r requirements.txt

In [2]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader, PyPDFLoader
from langchain_community.llms import Ollama
from langchain_chroma import Chroma
from itertools import islice
import tqdm
from module import *

/opt/environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
emb_model = "baai_small"
llm_model = "llama3"

## 2. Carga de datos 

In [4]:
documents_loader = PyPDFDirectoryLoader("./data_testing/test2")
documents = documents_loader.load()
print(f"Loaded {len(documents)} documents")

Loaded 306 documents


## 3. Creación de la base de datos vectorial

In [5]:
# Load the existing database.
db = Chroma(
    collection_name="test2",
    persist_directory="./database",
    embedding_function=get_embedding_function(emb_model)
)

# Adding documents with a progress bar
with tqdm.tqdm(total=len(documents), desc="Adding documents") as pbar:
    for document in documents:
        db.add_documents([document])
        pbar.update(1)

/opt/environment/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/opt/environment/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Adding documents: 100%|██████████| 306/306 [00:16<00:00, 18.28it/s]


## 4. Preguntas de interés

Formulamos las preguntas y buscamos en la base de datos los chunks que nos ofrezcan mejor contexto para responderlas.

In [6]:
questions_en = [
    "What is the clause that specifies the conditions for the rescission of the contract in the Arras Contract?",
    "What are the job functions described in the Indefinite Employment Contract?",
    "What penalties are applied if the buyer does not meet the conditions of the Arras Contract?",
]

questions_es = [
    "¿Cuál es la cláusula que especifica las condiciones para la rescisión del contrato en el Contrato de Arras?",
    "¿Cuáles son las funciones laborales descritas en el Contrato de Trabajo Indefinido?",
    "¿Qué sanciones se aplican si la parte compradora no cumple con las condiciones del Contrato de Arras?",
]

questions_cat = [
    "Quina és la clàusula que especifica les condicions per a la rescissió del contracte en el Contracte d'Arres?",
    "Quines són les funcions laborals descrites en el Contracte de Treball Indefinit?",
    "Quines sancions s'apliquen si la part compradora no compleix amb les condicions del Contracte d'Arres?",
]

questions = questions_en + questions_es + questions_cat

In [7]:
contexts = []

for question in questions:
    # Get the top 5 most relevant documents
    results = db.similarity_search_with_score(question, k=5)

    # Make a list of the contexts
    question_contexts = []
    for doc, _score in results:
        question_contexts.append(doc.page_content)

    # Append the context sub-list to the list of contexts
    contexts.append(question_contexts)

print(contexts)

[['    \n \n35 \n \n4.12.2. Despeses  \nLes despeses de personal i altres despeses de gestió ordinària es reconeixen en el compte del resultat \nen el  moment que merita l’obligació, amb independència del moment en què es produeixi el corrent \nmonetari que se’n deriva.  \n \n4.13. Provisions i contingències  \nEs reconeix una provisió quan l’Ajuntament té una obligació present (legal, contractual o implícita) \nresultat d’un succés passat que, probablement, implicarà el lliurament d’uns recursos que es poden \nestimar de forma fiable.  \nLes provisions es troben valorades pel valor actual dels desemborsaments que s’espera que seran \nnecessaris per liquidar l’obligació, actualit zats a una taxa de descompte equivalent al tipus de deute \npúblic per un venciment anàleg. Els ajustaments en la provisió amb motiu de la seva actualització es reconeixen com una despesa financera a mesura que es van reportant. Quan el seu venciment és inferior o igual a un any i l’efecte financer no és signi

Ahora creamos las respuestas esperadas para cada pregunta.

In [8]:
answers_en = [
    "In the Arras Contract, the clause that specifies the conditions for the rescission of the contract is the third clause",
    "The worker will provide services as [job title], included in the professional group of [professional group], for the performance of [job functions] in accordance with the professional classification system in force in the company",
    "The buyer may terminate this contract, agreeing to forfeit the amounts paid as penalty deposits.",
]

answers_es = [
    "En el Contrato de Arras, la cláusula que especifica las condiciones para la rescisión del contrato es la tercera cláusula",
    "El/la trabajador/a prestará sus servicios como [puesto], incluido en el grupo profesional de [grupo profesional], para la realización de las funciones de [funciones laborales] de acuerdo con el sistema de clasificación profesional vigente en la empresa",
    "La parte compradora podrá resolver el presente contrato, aviniéndose a perder las cantidades que ha entregado como arras penitenciales.",
]

answers_cat = [
    "En el Contracte d'Arres, la clàusula que especifica les condicions per a la rescissió del contracte és la tercera clàusula",
    "El/la treballador/a prestarà els seus serveis com [lloc de treball], inclosos en el grup professional de [grup professional], per a la realització de les funcions de [funcions laborals] d'acord amb el sistema de classificació professional vigent a l'empresa",
    "La part compradora podrà resoldre el present contracte, convenint-se a perdre les quantitats que ha entregat com a arres penitencials.",
]

answers = answers_en + answers_es + answers_cat
answers

['In the Arras Contract, the clause that specifies the conditions for the rescission of the contract is the third clause',
 'The worker will provide services as [job title], included in the professional group of [professional group], for the performance of [job functions] in accordance with the professional classification system in force in the company',
 'The buyer may terminate this contract, agreeing to forfeit the amounts paid as penalty deposits.',
 'En el Contrato de Arras, la cláusula que especifica las condiciones para la rescisión del contrato es la tercera cláusula',
 'El/la trabajador/a prestará sus servicios como [puesto], incluido en el grupo profesional de [grupo profesional], para la realización de las funciones de [funciones laborales] de acuerdo con el sistema de clasificación profesional vigente en la empresa',
 'La parte compradora podrá resolver el presente contrato, aviniéndose a perder las cantidades que ha entregado como arras penitenciales.',
 "En el Contracte d

Finalmente creamos el DataFrame para evaluar los modelos.

In [9]:
data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

## 5. Evaluación de los modelos

In [10]:
embeddings = get_embedding_function(emb_model)
llm = Ollama(model=llm_model)

In [ ]:
import nest_asyncio
from ragas.run_config import RunConfig


nest_asyncio.apply()

try:
    score = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=False,
        run_config=RunConfig(
            #max_retries=30, # Default is 10
            #max_wait=180, # Default is 60
            #max_workers=64 # Default is 16
        )
    )
except Exception as e:
    print(f"An error ocurred: {e}")

df_score = score.to_pandas()
print(df_score)

Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
df_score.head(10)